In [1]:
import torch
from Model_B import B13Net, B13NetV1, UNet

from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

input = torch.randn(2, 16, 16, 16, 16)

model = B13NetV1(output_size=3).to(device)
writer = SummaryWriter("runs/Model_B")
print(summary(model, input_size=(2, 16, 16, 16, 16)))



----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1       [-1, 32, 16, 16, 16]          13,856
              ReLU-2       [-1, 32, 16, 16, 16]               0
            Conv3d-3       [-1, 32, 16, 16, 16]          27,680
              ReLU-4       [-1, 32, 16, 16, 16]               0
            Conv3d-5       [-1, 32, 16, 16, 16]          13,856
              ReLU-6       [-1, 32, 16, 16, 16]               0
            Conv3d-7       [-1, 32, 16, 16, 16]          27,680
              ReLU-8       [-1, 32, 16, 16, 16]               0
            Conv3d-9       [-1, 32, 16, 16, 16]          13,856
             ReLU-10       [-1, 32, 16, 16, 16]               0
           Conv3d-11       [-1, 32, 16, 16, 16]          27,680
             ReLU-12       [-1, 32, 16, 16, 16]               0
           Conv3d-13       [-1, 32, 16, 16, 16]          13,856
             ReLU-14       [-1, 32, 16,

In [4]:
model2 = UNet(in_channels=1, out_channels=1, depth=4, feat=40, final_activation=None).to(device)
print(summary(model2, input_size=(1, 256, 256)))



----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 40, 256, 256]             400
              ReLU-2         [-1, 40, 256, 256]               0
            Conv2d-3         [-1, 40, 256, 256]          14,440
              ReLU-4         [-1, 40, 256, 256]               0
         MaxPool2d-5         [-1, 40, 128, 128]               0
            Conv2d-6         [-1, 80, 128, 128]          28,880
              ReLU-7         [-1, 80, 128, 128]               0
            Conv2d-8         [-1, 80, 128, 128]          57,680
              ReLU-9         [-1, 80, 128, 128]               0
        MaxPool2d-10           [-1, 80, 64, 64]               0
           Conv2d-11          [-1, 160, 64, 64]         115,360
             ReLU-12          [-1, 160, 64, 64]               0
           Conv2d-13          [-1, 160, 64, 64]         230,560
             ReLU-14          [-1, 160,

In [5]:
from graphviz import Digraph
import torch
from torch.autograd import Variable


def make_dot(var, params):
    """ Produces Graphviz representation of PyTorch autograd graph
    
    Blue nodes are the Variables that require grad, orange are Tensors
    saved for backward in torch.autograd.Function
    
    Args:
        var: output Variable
        params: dict of (name, Variable) to add names to node that
            require grad (TODO: make optional)
    """
    param_map = {id(v): k for k, v in params.items()}
    print(param_map)
    
    node_attr = dict(style='filled',
                     shape='box',
                     align='left',
                     fontsize='12',
                     ranksep='0.1',
                     height='0.2')
    dot = Digraph(node_attr=node_attr, graph_attr=dict(size="12,12"))
    seen = set()
    
    def size_to_str(size):
        return '('+(', ').join(['%d'% v for v in size])+')'

    def add_nodes(var):
        if var not in seen:
            if torch.is_tensor(var):
                dot.node(str(id(var)), size_to_str(var.size()), fillcolor='orange')
            elif hasattr(var, 'variable'):
                u = var.variable
                node_name = '%s\n %s' % (param_map.get(id(u)), size_to_str(u.size()))
                dot.node(str(id(var)), node_name, fillcolor='lightblue')
            else:
                dot.node(str(id(var)), str(type(var).__name__))
            seen.add(var)
            if hasattr(var, 'next_functions'):
                for u in var.next_functions:
                    if u[0] is not None:
                        dot.edge(str(id(u[0])), str(id(var)))
                        add_nodes(u[0])
            if hasattr(var, 'saved_tensors'):
                for t in var.saved_tensors:
                    dot.edge(str(id(t)), str(id(var)))
                    add_nodes(t)
    add_nodes(var.grad_fn)
    return dot

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
g = make_dot(model, model.state_dict())
g.view()


{140221317902032: 'encoder.0.0.weight', 140221317902272: 'encoder.0.0.bias', 140221317896592: 'encoder.0.2.weight', 140221317895232: 'encoder.0.2.bias', 140221317894032: 'encoder.1.0.weight', 140221317891952: 'encoder.1.0.bias', 140221317890112: 'encoder.1.2.weight', 140221510190640: 'encoder.1.2.bias', 140221355192976: 'encoderB.0.0.weight', 140221319511824: 'encoderB.0.0.bias', 140221317662736: 'encoderB.0.2.weight', 140221317668016: 'encoderB.0.2.bias', 140221317673216: 'encoderB.1.0.weight', 140221317671296: 'encoderB.1.0.bias', 140221317676656: 'encoderB.1.2.weight', 140221317676416: 'encoderB.1.2.bias'}


AttributeError: 'B13NetV1' object has no attribute 'grad_fn'